In [ ]:
class Net(nn.Module):
    def __init__(self, D_in: int, hidden: int, D_out: int, num_tasks: int, dropout=0.1):
        '''
        D_in (int): input dimension
        hidden (int): number of hidden layers
        D_out (int): output dimension or the number of classes
        num_tasks (int): number of tasks to predict
        dropout (float): percentage of neurons to be masked
        '''
        super().__init__()
        self.num_tasks = num_tasks
        self.bottlenet = nn.Linear(D_in, hidden)
        self.layers = nn.ModuleList([nn.Linear(hidden, D_out) for i in range(num_tasks)])
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        yhat = torch.tensor([])
        x = self.dropout(self.bottlenet(x))
        for task in self.layers:
            y = torch.sigmoid(self.dropout(F.relu(task(x))))
            yhat = torch.cat((yhat, y), 1)
        return yhat
    
model = Net(D_in, hidden, D_out, num_tasks, dropout)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def criterion(yhat, y):
    mask = [y != -1]
    y, yhat = y[mask], yhat[mask]
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(yhat, y)
    return loss

from torch_lr_finder import LRFinder
lr_finder = LRFinder(model, optimizer, criterion)
lr_finder.range_test(train_loader, start_lr = 0.0001, end_lr=0.03, num_iter=100)
lr_finder.plot(suggest_lr=True) # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state.